In [3]:
from bs4 import BeautifulSoup
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
import random

In [4]:
# ✅ Configuration Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Mode sans interface graphique
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

# ✅ Initialiser WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ✅ Ouvrir la page Fragrantica
url = "https://www.fragrantica.com/designers/"
driver.get(url)

# ✅ Attendre le chargement initial
time.sleep(5)  # Attendre quelques secondes pour que la page charge complètement

# ✅ Extraire les designers
designers_data = []

# Récupérer tous les blocs contenant les marques
designers_elements = driver.find_elements(By.CLASS_NAME, "designerlist")

for designer in designers_elements:
    try:
        # Trouver le lien et le nom
        link_element = designer.find_element(By.TAG_NAME, "a")
        brand_name = link_element.text.strip()
        brand_link = link_element.get_attribute("href")
        designers_data.append([brand_name, brand_link])
    except Exception as e:
        print(f"Erreur sur un élément : {e}")

# ✅ Fermer le navigateur
driver.quit()

# ✅ Créer un DataFrame Pandas
df_designers = pd.DataFrame(designers_data, columns=['Nom de la Marque', 'Lien'])


In [5]:
df_designers

,Nom de la Marque,Lien
0,Acqua di Parma,https://www.fragrantica.com/designers/Acqua-di...
1,Afnan,https://www.fragrantica.com/designers/Afnan.html
2,Al Haramain Perfumes,https://www.fragrantica.com/designers/Al-Haram...
3,,https://www.fragrantica.com/designers/Al-Rehab...
4,,https://www.fragrantica.com/designers/Amouage....
...,...,...
115,,https://www.fragrantica.com/designers/Xerjoff....
116,,https://www.fragrantica.com/designers/Yves-Roc...
117,,https://www.fragrantica.com/designers/Yves-Sai...
118,,https://www.fragrantica.com/designers/Zara.html


In [6]:
def scrape_perfumes_from_designer(designer_url):
    """
    Récupère tous les parfums d'un designer sur Fragrantica en utilisant Selenium + BeautifulSoup.
    
    :param designer_url: URL de la page du designer sur Fragrantica.
    :return: DataFrame avec les colonnes ['Marque', 'Parfum', 'Lien du Parfum'].
    """
    # ✅ Configuration Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Mode sans interface graphique
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

    # ✅ Initialiser WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # ✅ Ouvrir la page du designer
    driver.get(designer_url)

    # ✅ Attendre le chargement de la page
    time.sleep(5)

    # ✅ Récupérer le contenu HTML de la page
    html = driver.page_source

    # ✅ Fermer le navigateur
    driver.quit()

    # ✅ Parser le HTML avec BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # ✅ Récupérer le nom du designer (H1)
    brand_name = soup.find("h1").text.replace(" perfumes and colognes", "").strip() if soup.find("h1") else "Inconnu"

    # ✅ Récupérer tous les parfums
    perfumes_data = []

    # Trouver tous les parfums listés sous <h3> avec un <a>
    perfume_elements = soup.select("h3 a")

    for element in perfume_elements:
        perfume_name = element.text.strip()
        perfume_link = "https://www.fragrantica.com" + element.get("href")
        
        if perfume_name:  # Vérifier si le nom est valide
            perfumes_data.append([brand_name, perfume_name, perfume_link])

    # ✅ Créer un DataFrame Pandas
    df = pd.DataFrame(perfumes_data, columns=['Marque', 'Parfum', 'Lien du Parfum'])

    return df

In [9]:
scrape_perfumes_from_designer('https://www.fragrantica.com/designers/Dior.html')

,Marque,Parfum,Lien du Parfum
0,Dior,Bonne Étoile Baby Dior,https://www.fragrantica.com/perfume/Dior/Bonne...
1,Dior,Chris 1947,https://www.fragrantica.com/perfume/Dior/Chris...
2,Dior,Dior Dior,https://www.fragrantica.com/perfume/Dior/Dior-...
3,Dior,"Dior Me, Dior Me Not",https://www.fragrantica.com/perfume/Dior/Dior-...
4,Dior,Dior Star,https://www.fragrantica.com/perfume/Dior/Dior-...
...,...,...,...
285,Dior,Fahrenheit Cologne,https://www.fragrantica.com/perfume/Dior/Fahre...
286,Dior,Hypnotic Poison Collector Rubis,https://www.fragrantica.com/perfume/Dior/Hypno...
287,Dior,Poison Eau de Cologne,https://www.fragrantica.com/perfume/Dior/Poiso...
288,Dior,Poison Parfum,https://www.fragrantica.com/perfume/Dior/Poiso...


In [ ]:
all_perfumes = []

errors = []

# ✅ Boucler sur chaque designer
for index, row in df_designers.iterrows():
    designer_url = row['Lien']  # Assurez-vous que la colonne contient les URLs valides
    
    try:
        print(f"🔄 Scraping {index+1}/{len(df_designers)} : {designer_url}")
        
        # Récupérer les parfums du designer
        df_perfumes = scrape_perfumes_from_designer(designer_url)
        
        # Vérifier si des parfums ont été récupérés
        if not df_perfumes.empty:
            all_perfumes.append(df_perfumes)
            print(f"✅ {len(df_perfumes)} parfums récupérés pour {designer_url}")
        else:
            print(f"⚠️ Aucune donnée trouvée pour {designer_url}")
            errors.append(designer_url)
        
    except Exception as e:
        print(f"❌ Erreur avec {designer_url} : {e}")
        errors.append(designer_url)

    # Pause pour éviter de se faire bloquer
    sleep_time = random.uniform(5, 10)
    time.sleep(sleep_time)

# ✅ Fusionner tous les DataFrames en un seul
if all_perfumes:
    df_final = pd.concat(all_perfumes, ignore_index=True)

    # ✅ Sauvegarder la BDD en CSV
    df_final.to_csv("parfums_database.csv", index=False)

print("✅ Base de données des parfums enregistrée sous 'parfums_database.csv'")

if errors:
    pd.DataFrame(errors, columns=["Lien"]).to_csv("erreurs_designers.csv", index=False)
    print(f"⚠️ {len(errors)} designers n'ont pas pu être scrapés. Liste enregistrée sous 'erreurs_designers.csv'")

🔄 Scraping 1/120 : https://www.fragrantica.com/designers/Acqua-di-Parma.html
✅ 92 parfums récupérés pour https://www.fragrantica.com/designers/Acqua-di-Parma.html
🔄 Scraping 2/120 : https://www.fragrantica.com/designers/Afnan.html
✅ 125 parfums récupérés pour https://www.fragrantica.com/designers/Afnan.html
🔄 Scraping 3/120 : https://www.fragrantica.com/designers/Al-Haramain-Perfumes.html
✅ 331 parfums récupérés pour https://www.fragrantica.com/designers/Al-Haramain-Perfumes.html
🔄 Scraping 4/120 : https://www.fragrantica.com/designers/Al-Rehab.html
✅ 248 parfums récupérés pour https://www.fragrantica.com/designers/Al-Rehab.html
🔄 Scraping 5/120 : https://www.fragrantica.com/designers/Amouage.html
✅ 138 parfums récupérés pour https://www.fragrantica.com/designers/Amouage.html
🔄 Scraping 6/120 : https://www.fragrantica.com/designers/Ariana-Grande.html
✅ 26 parfums récupérés pour https://www.fragrantica.com/designers/Ariana-Grande.html
🔄 Scraping 7/120 : https://www.fragrantica.com/desig

Exception ignored in: <function Service.__del__ at 0x0000016C40FFCCC0>
Traceback (most recent call last):
  File "C:\Users\hacho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\selenium\webdriver\common\service.py", line 200, in __del__
    self.stop()
  File "C:\Users\hacho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\selenium\webdriver\common\service.py", line 157, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\hacho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\selenium\webdriver\common\service.py", line 137, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\urllib\request.py", line 216, in urlop

⚠️ Aucune donnée trouvée pour https://www.fragrantica.com/designers/Diptyque.html


In [10]:
data = pd.read_csv('parfums_database.csv')
data

,Marque,Parfum,Lien du Parfum
0,Acqua di Parma,Acqua di Parma Colonia,https://www.fragrantica.com/perfume/Acqua-di-P...
1,Acqua di Parma,Acqua di Parma Colonia Assoluta,https://www.fragrantica.com/perfume/Acqua-di-P...
2,Acqua di Parma,Acqua di Parma Colonia Assoluta Edizione Riviera,https://www.fragrantica.com/perfume/Acqua-di-P...
3,Acqua di Parma,Acqua di Parma Colonia Assoluta Edizione Speci...,https://www.fragrantica.com/perfume/Acqua-di-P...
4,Acqua di Parma,Acqua di Parma Colonia Designer Edition,https://www.fragrantica.com/perfume/Acqua-di-P...
...,...,...,...
5292,DS&Durga,40 Million Year Old Amber,https://www.fragrantica.com/perfume/DS-Durga/4...
5293,DS&Durga,Peanut,https://www.fragrantica.com/perfume/DS-Durga/P...
5294,DS&Durga,Roman Fruit Sellers,https://www.fragrantica.com/perfume/DS-Durga/R...
5295,DS&Durga,Roman Ruin Cypress,https://www.fragrantica.com/perfume/DS-Durga/R...


In [ ]:
try:
    df_existing = pd.read_csv("parfums_database.csv")
    existing_brands = df_existing['Marque'].unique().tolist()  # Liste des marques déjà scrapées
    print(f"🔄 {len(existing_brands)} designers déjà enregistrés.")
except FileNotFoundError:
    df_existing = pd.DataFrame(columns=['Marque', 'Parfum', 'Lien du Parfum'])
    existing_brands = []
    print("🚀 Aucune base existante trouvée, on commence de zéro.")

# ✅ Continuer le scraping à partir du 29ᵉ designer
all_perfumes = []
errors = []

for index, row in df_designers.iloc[28:].iterrows():  # Commencer à partir du 29e designer
    designer_url = row['Lien']  # Lien du designer

    try:
        print(f"🔄 Scraping {index+1}/{len(df_designers)} : {designer_url}")

        # Récupérer les parfums du designer
        df_perfumes = scrape_perfumes_from_designer(designer_url)

        # Vérifier si la marque est déjà dans la base
        if df_perfumes.empty or df_perfumes['Marque'].iloc[0] in existing_brands:
            print(f"⚠️ Marque déjà scrapée ou aucun parfum trouvé : {designer_url}")
            continue

        # Ajouter les parfums au DataFrame
        all_perfumes.append(df_perfumes)
        print(f"✅ {len(df_perfumes)} parfums récupérés pour {designer_url}")

        # Mettre à jour la liste des marques déjà scrapées
        existing_brands.append(df_perfumes['Marque'].iloc[0])

    except Exception as e:
        print(f"❌ Erreur avec {designer_url} : {e}")
        errors.append(designer_url)

    # ✅ Pause aléatoire pour éviter le blocage
    sleep_time = random.uniform(5, 10)
    time.sleep(sleep_time)

# ✅ Fusionner avec la base de données existante
if all_perfumes:
    df_new = pd.concat(all_perfumes, ignore_index=True)
    df_final = pd.concat([df_existing, df_new], ignore_index=True)

    # ✅ Sauvegarder la BDD mise à jour en CSV
    df_final.to_csv("parfums_database.csv", index=False)

    print("✅ Base de données mise à jour enregistrée sous 'parfums_database.csv'")

# ✅ Sauvegarder les erreurs si besoin
if errors:
    pd.DataFrame(errors, columns=["Lien"]).to_csv("erreurs_designers.csv", index=False)
    print(f"⚠️ {len(errors)} designers n'ont pas pu être scrapés. Liste enregistrée sous 'erreurs_designers.csv'")

🔄 29 designers déjà enregistrés.
🔄 Scraping 29/120 : https://www.fragrantica.com/designers/DS-Durga.html
⚠️ Marque déjà scrapée ou aucun parfum trouvé : https://www.fragrantica.com/designers/DS-Durga.html
🔄 Scraping 30/120 : https://www.fragrantica.com/designers/Davidoff.html
✅ 107 parfums récupérés pour https://www.fragrantica.com/designers/Davidoff.html
🔄 Scraping 31/120 : https://www.fragrantica.com/designers/Demeter-Fragrance.html
✅ 431 parfums récupérés pour https://www.fragrantica.com/designers/Demeter-Fragrance.html
🔄 Scraping 32/120 : https://www.fragrantica.com/designers/Dior.html
✅ 290 parfums récupérés pour https://www.fragrantica.com/designers/Dior.html
🔄 Scraping 33/120 : https://www.fragrantica.com/designers/Diptyque.html
✅ 90 parfums récupérés pour https://www.fragrantica.com/designers/Diptyque.html
🔄 Scraping 34/120 : https://www.fragrantica.com/designers/Dolce-Gabbana.html
✅ 122 parfums récupérés pour https://www.fragrantica.com/designers/Dolce-Gabbana.html
🔄 Scraping 

NameError: name 'tools' is not defined

In [13]:
data_2 = pd.read_csv('parfums_database.csv')
data_2

,Marque,Parfum,Lien du Parfum
0,Acqua di Parma,Acqua di Parma Colonia,https://www.fragrantica.com/perfume/Acqua-di-P...
1,Acqua di Parma,Acqua di Parma Colonia Assoluta,https://www.fragrantica.com/perfume/Acqua-di-P...
2,Acqua di Parma,Acqua di Parma Colonia Assoluta Edizione Riviera,https://www.fragrantica.com/perfume/Acqua-di-P...
3,Acqua di Parma,Acqua di Parma Colonia Assoluta Edizione Speci...,https://www.fragrantica.com/perfume/Acqua-di-P...
4,Acqua di Parma,Acqua di Parma Colonia Designer Edition,https://www.fragrantica.com/perfume/Acqua-di-P...
...,...,...,...
10531,Maison Alhambra,Zeno,https://www.fragrantica.com/perfume/Maison-Alh...
10532,Maison Alhambra,Kismet Angel,https://www.fragrantica.com/perfume/Maison-Alh...
10533,Maison Alhambra,Kismet for Men,https://www.fragrantica.com/perfume/Maison-Alh...
10534,Maison Alhambra,Kismet for Women,https://www.fragrantica.com/perfume/Maison-Alh...


In [14]:
try:
    df_existing = pd.read_csv("parfums_database.csv")
    existing_brands = df_existing['Marque'].unique().tolist()  # Liste des marques déjà scrapées
    print(f"🔄 {len(existing_brands)} designers déjà enregistrés.")
except FileNotFoundError:
    df_existing = pd.DataFrame(columns=['Marque', 'Parfum', 'Lien du Parfum'])
    existing_brands = []
    print("🚀 Aucune base existante trouvée, on commence de zéro.")

# ✅ Continuer le scraping à partir du 72ᵉ designer
all_perfumes = []
errors = []

for index, row in df_designers.iloc[71:].iterrows():  # Commencer à partir du 72e designer
    designer_url = row['Lien']  # Lien du designer

    try:
        print(f"🔄 Scraping {index+1}/{len(df_designers)} : {designer_url}")

        # Récupérer les parfums du designer
        df_perfumes = scrape_perfumes_from_designer(designer_url)

        # Vérifier si la marque est déjà dans la base
        if df_perfumes.empty or df_perfumes['Marque'].iloc[0] in existing_brands:
            print(f"⚠️ Marque déjà scrapée ou aucun parfum trouvé : {designer_url}")
            continue

        # Ajouter les parfums au DataFrame
        all_perfumes.append(df_perfumes)
        print(f"✅ {len(df_perfumes)} parfums récupérés pour {designer_url}")

        # Mettre à jour la liste des marques déjà scrapées
        existing_brands.append(df_perfumes['Marque'].iloc[0])

    except Exception as e:
        print(f"❌ Erreur avec {designer_url} : {e}")
        errors.append(designer_url)

    # ✅ Pause aléatoire pour éviter le blocage
    sleep_time = random.uniform(5, 10)
    time.sleep(sleep_time)

# ✅ Fusionner avec la base de données existante
if all_perfumes:
    df_new = pd.concat(all_perfumes, ignore_index=True)
    df_final = pd.concat([df_existing, df_new], ignore_index=True)

    # ✅ Sauvegarder la BDD mise à jour en CSV
    df_final.to_csv("parfums_database.csv", index=False)

    print("✅ Base de données mise à jour enregistrée sous 'parfums_database.csv'")

# ✅ Sauvegarder les erreurs si besoin
if errors:
    pd.DataFrame(errors, columns=["Lien"]).to_csv("erreurs_designers.csv", index=False)
    print(f"⚠️ {len(errors)} designers n'ont pas pu être scrapés. Liste enregistrée sous 'erreurs_designers.csv'")

🔄 70 designers déjà enregistrés.
🔄 Scraping 72/120 : https://www.fragrantica.com/designers/Maison-Crivelli.html
✅ 20 parfums récupérés pour https://www.fragrantica.com/designers/Maison-Crivelli.html
🔄 Scraping 73/120 : https://www.fragrantica.com/designers/Maison-Francis-Kurkdjian.html
✅ 60 parfums récupérés pour https://www.fragrantica.com/designers/Maison-Francis-Kurkdjian.html
🔄 Scraping 74/120 : https://www.fragrantica.com/designers/Maison-Martin-Margiela.html
✅ 35 parfums récupérés pour https://www.fragrantica.com/designers/Maison-Martin-Margiela.html
🔄 Scraping 75/120 : https://www.fragrantica.com/designers/Mancera.html
✅ 91 parfums récupérés pour https://www.fragrantica.com/designers/Mancera.html
🔄 Scraping 76/120 : https://www.fragrantica.com/designers/Marc-Jacobs.html
✅ 121 parfums récupérés pour https://www.fragrantica.com/designers/Marc-Jacobs.html
🔄 Scraping 77/120 : https://www.fragrantica.com/designers/Marc-Antoine-Barrois.html
✅ 6 parfums récupérés pour https://www.fragr

In [15]:
pd.read_csv('parfums_database.csv')

,Marque,Parfum,Lien du Parfum
0,Acqua di Parma,Acqua di Parma Colonia,https://www.fragrantica.com/perfume/Acqua-di-P...
1,Acqua di Parma,Acqua di Parma Colonia Assoluta,https://www.fragrantica.com/perfume/Acqua-di-P...
2,Acqua di Parma,Acqua di Parma Colonia Assoluta Edizione Riviera,https://www.fragrantica.com/perfume/Acqua-di-P...
3,Acqua di Parma,Acqua di Parma Colonia Assoluta Edizione Speci...,https://www.fragrantica.com/perfume/Acqua-di-P...
4,Acqua di Parma,Acqua di Parma Colonia Designer Edition,https://www.fragrantica.com/perfume/Acqua-di-P...
...,...,...,...
13953,Salvatore Ferragamo,Uomo Salvatore Ferragamo,https://www.fragrantica.com/perfume/Salvatore-...
13954,Salvatore Ferragamo,Uomo Salvatore Ferragamo Casual Life,https://www.fragrantica.com/perfume/Salvatore-...
13955,Salvatore Ferragamo,Uomo Salvatore Ferragamo Limited Edition,https://www.fragrantica.com/perfume/Salvatore-...
13956,Salvatore Ferragamo,Uomo Salvatore Ferragamo Signature,https://www.fragrantica.com/perfume/Salvatore-...


In [16]:
try:
    df_existing = pd.read_csv("parfums_database.csv")
    existing_brands = df_existing['Marque'].unique().tolist()  # Liste des marques déjà scrapées
    print(f"🔄 {len(existing_brands)} designers déjà enregistrés.")
except FileNotFoundError:
    df_existing = pd.DataFrame(columns=['Marque', 'Parfum', 'Lien du Parfum'])
    existing_brands = []
    print("🚀 Aucune base existante trouvée, on commence de zéro.")

# ✅ Continuer le scraping à partir du 102ᵉ designer
all_perfumes = []
errors = []

for index, row in df_designers.iloc[101:].iterrows():  # Commencer à partir du 102e designer
    designer_url = row['Lien']  # Lien du designer

    try:
        print(f"🔄 Scraping {index+1}/{len(df_designers)} : {designer_url}")

        # Récupérer les parfums du designer
        df_perfumes = scrape_perfumes_from_designer(designer_url)

        # Vérifier si la marque est déjà dans la base
        if df_perfumes.empty or df_perfumes['Marque'].iloc[0] in existing_brands:
            print(f"⚠️ Marque déjà scrapée ou aucun parfum trouvé : {designer_url}")
            continue

        # Ajouter les parfums au DataFrame
        all_perfumes.append(df_perfumes)
        print(f"✅ {len(df_perfumes)} parfums récupérés pour {designer_url}")

        # Mettre à jour la liste des marques déjà scrapées
        existing_brands.append(df_perfumes['Marque'].iloc[0])

    except Exception as e:
        print(f"❌ Erreur avec {designer_url} : {e}")
        errors.append(designer_url)

    # ✅ Pause aléatoire pour éviter le blocage
    sleep_time = random.uniform(5, 10)
    time.sleep(sleep_time)

# ✅ Fusionner avec la base de données existante
if all_perfumes:
    df_new = pd.concat(all_perfumes, ignore_index=True)
    df_final = pd.concat([df_existing, df_new], ignore_index=True)

    # ✅ Sauvegarder la BDD mise à jour en CSV
    df_final.to_csv("parfums_database.csv", index=False)

    print("✅ Base de données mise à jour enregistrée sous 'parfums_database.csv'")

# ✅ Sauvegarder les erreurs si besoin
if errors:
    pd.DataFrame(errors, columns=["Lien"]).to_csv("erreurs_designers.csv", index=False)
    print(f"⚠️ {len(errors)} designers n'ont pas pu être scrapés. Liste enregistrée sous 'erreurs_designers.csv'")

🔄 100 designers déjà enregistrés.
🔄 Scraping 102/120 : https://www.fragrantica.com/designers/Serge-Lutens.html
✅ 107 parfums récupérés pour https://www.fragrantica.com/designers/Serge-Lutens.html
🔄 Scraping 103/120 : https://www.fragrantica.com/designers/Sol-de-Janeiro.html
✅ 23 parfums récupérés pour https://www.fragrantica.com/designers/Sol-de-Janeiro.html
🔄 Scraping 104/120 : https://www.fragrantica.com/designers/Stephane-Humbert-Lucas-777.html
✅ 31 parfums récupérés pour https://www.fragrantica.com/designers/Stephane-Humbert-Lucas-777.html
🔄 Scraping 105/120 : https://www.fragrantica.com/designers/Swiss-Arabian.html
✅ 242 parfums récupérés pour https://www.fragrantica.com/designers/Swiss-Arabian.html
🔄 Scraping 106/120 : https://www.fragrantica.com/designers/The-Dua-Brand.html
✅ 1673 parfums récupérés pour https://www.fragrantica.com/designers/The-Dua-Brand.html
🔄 Scraping 107/120 : https://www.fragrantica.com/designers/Tiziana-Terenzi.html
✅ 119 parfums récupérés pour https://www.

In [17]:
df_final = pd.read_csv('parfums_database.csv')
df_final

,Marque,Parfum,Lien du Parfum
0,Acqua di Parma,Acqua di Parma Colonia,https://www.fragrantica.com/perfume/Acqua-di-P...
1,Acqua di Parma,Acqua di Parma Colonia Assoluta,https://www.fragrantica.com/perfume/Acqua-di-P...
2,Acqua di Parma,Acqua di Parma Colonia Assoluta Edizione Riviera,https://www.fragrantica.com/perfume/Acqua-di-P...
3,Acqua di Parma,Acqua di Parma Colonia Assoluta Edizione Speci...,https://www.fragrantica.com/perfume/Acqua-di-P...
4,Acqua di Parma,Acqua di Parma Colonia Designer Edition,https://www.fragrantica.com/perfume/Acqua-di-P...
...,...,...,...
19313,Zoologist Perfumes,Dodo Edition 2020,https://www.fragrantica.com/perfume/Zoologist-...
19314,Zoologist Perfumes,Dragonfly,https://www.fragrantica.com/perfume/Zoologist-...
19315,Zoologist Perfumes,Macaque,https://www.fragrantica.com/perfume/Zoologist-...
19316,Zoologist Perfumes,Panda,https://www.fragrantica.com/perfume/Zoologist-...
